# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name:Womenker Karto     [Github](https://github.com/wkarto/Module4_json-sentiment)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

## Question 1:

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [1]:
import requests
import json

response = requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul')
result = response.json()

with open("lyrics.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=4)


## Question 2:

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [2]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

with open("lyrics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

lyrics = data.get("lyrics", "")

# Load spaCy and add TextBlob pipe
nlp = spacy.load("en_core_web_sm")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")

# Perform sentiment analysis
doc = nlp(lyrics)
polarity = doc._.blob.polarity
print(f"\nPolarity Score: {polarity:.3f}")

# The result here is 0.045, which is slightly above neutral.
# Therefore, the lyrics have a **slightly positive connotation**.


Polarity Score: 0.045


## Question 3:

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [3]:
import requests
import json

def save_lyrics(artist, song, filename):
    """Fetch lyrics from lyrics.ovh API and save to a JSON file."""
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    response = requests.get(url)
    
    if response.status_code == 200:
        lyrics_data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(lyrics_data, f, indent=4)
        print(f"Lyrics for '{song}' by {artist} saved to '{filename}'")
    else:
        print(f"Failed to fetch lyrics for '{song}' by {artist}. Status Code: {response.status_code}")

save_lyrics("Coldplay", "Yellow", "yellow.json")
save_lyrics("Queen", "Bohemian Rhapsody", "bohemian.json")
save_lyrics("Eminem", "Lose Yourself", "lose_yourself.json")
save_lyrics("Linkin Park", "Numb", "numb.json")


Failed to fetch lyrics for 'Yellow' by Coldplay. Status Code: 504
Lyrics for 'Bohemian Rhapsody' by Queen saved to 'bohemian.json'
Lyrics for 'Lose Yourself' by Eminem saved to 'lose_yourself.json'
Lyrics for 'Numb' by Linkin Park saved to 'numb.json'


## Question 4:

4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [4]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load("en_core_web_sm")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob", last=True)

def get_lyrics_sentiment(filename):
    """Load lyrics from a JSON file and return polarity score."""
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        lyrics = data.get("lyrics", "")
        doc = nlp(lyrics)
        return doc._.blob.polarity

# Function call
songs = {
    "Bohemian Rhapsody": "bohemian.json",
    "Lose Yourself": "lose_yourself.json",
    "Numb": "numb.json"
}

for title, file in songs.items():
    polarity = get_lyrics_sentiment(file)
    print(f"{title} — Polarity Score: {polarity:.3f}")


Bohemian Rhapsody — Polarity Score: -0.003
Lose Yourself — Polarity Score: 0.154
Numb — Polarity Score: -0.049


## Reflection on Sentiment Scores

- **Bohemian Rhapsody (-0.003)**:  
  Almost neutral. Makes sense since the song has mixed emotions and shifting tones.

- **Lose Yourself (0.154)**:  
  Slightly positive. Surprising, but maybe because it’s motivational overall.

- **Numb (-0.049)**:  
  Slightly negative. Fits the theme of feeling lost and frustrated.

### Overall:
The scores mostly match the mood of the lyrics. Some are subtle, which makes sense since songs can be complex.
